# Class 9 (11.10.2021)

# Important OS concepts
* A Process is a program under execution.
* It consists of a Program Counter, Process Stack, Registers, Program Code, etc.
* A thread is a light weight process managed independently by the scheduler.
* Threads improve performance via parallelism and concurrency.
* Threads share data segment, code and files with other threads belonging to the same process.
* Threads have their own Registers, Stack and Program Counter.
* For simplicity, one can consider Threads to be a subset of a Processes.

In [1]:
def cube(num):
    print(f"Cube of the given number if {num ** 3}")

def square(num):
    print(f"Square of the given number if {num ** 2}")

## Thread Class in Python
To create a new thread, we create an object of the Thread class. It takes following arguments:
* ***Target***: Function to be executed by the thread
* ***Args***: Arguments to be passed to the target function
 
Useful class methods:
- ***start***: We use the **start** method of the Thread class to start the execution of a thread.
- ***join***: Once a new thread starts executing, the current program (the main thread) also continues to execute. In order to stop the execution of the current program until a thread's execution is complete, we use the **join** method. As a result, the main thread will wait for the completion of the other thread before executing the remaining statements.

In [2]:
# Run this multiple times to show how the threads compete for stdout
from threading import Thread

t1 = Thread(target = cube, args = (10,))  # Args has to be an iterable like a list or a tuple
t2 = Thread(target = square, args = (5,))

# Starting the thread's execution
t1.start()
t2.start()

# This will be printed by the main thread
print("3 Threads are currently in execution") # Due to a race condition for stdout, the output will be jumbled

# Wait untill t1 and t2 complete execution
t1.join() # Main thread waits for this thread to finish execution (main thread is blocked)
t2.join() # Main thread is blocked untill t2 finishes

# This will be printed by the main thread
print("Both Completed") # There is no competition for stdout once all the threads have completed execution.

Cube of the given number if 1000
Square of the given number if 25
3 Threads are currently in execution
Both Completed


In [3]:
def print_loop_even(num):
    for i in range(num):
        print(f"{2*i} ==")


def print_loop_odd(num):
    for i in range(num):
        print(f"{2*i + 1} --")

In [4]:
# Run this multiple times to see how the threads compete for stdout
from threading import Thread

t1 = Thread(target = print_loop_even, args = (10,))  # Args has to be an iterable
t2 = Thread(target = print_loop_odd, args = (10,))

# Starting the thread's execution
t1.start()
t2.start()

# This will be printed by the main thread
print("3 Threads are currently in execution") # Due to a race condition for stdout, the output will be jumbled

# Wait untill t1 completes execution
t1.join() # Main thread waits for this thread to finish execution (main thread is blocked)
t2.join() # Main thread is blocked untill t2 finishes

print("Both Completed")

0 ==
2 ==
1 --
3 --4 ==
6 ==
8 ==
10 ==
5 --
7 --
3 Threads are currently in execution
9 --
11 --
13 --
15 --
17 --
19 --

12 ==
14 ==
16 ==
18 ==
Both Completed


## Concept of global (scope of a variable)
Inside a function, the local variables are given higher priority than the variables declared outside (global variables).

In [5]:
x = 45
def example():
    x = 1
    print(f"Inside function {x}")
    x = 2

example()
print(f"Outside function {x}")

Inside function 1
Outside function 45


In [6]:
x = 45
def example():
    x += 1
    print(f"Inside function {x}")
    x = 2

example()
print(f"Outside function {x}")

UnboundLocalError: local variable 'x' referenced before assignment

In [7]:
x = 45
def example():
    global x
    x += 1
    print(f"Inside function {x}")
    
example()
print(f"Outside function {x}")

Inside function 46
Outside function 46


In [8]:
# Showing thread collisions

globVar = 0

def incGlobal():
    global globVar
    globVar += 1

def threadTask():
    for _ in range(1000000):
        incGlobal()

def main():
    global globVar
    globVar = 0
    
    t1 = Thread(target = threadTask) # No arguments for the target function
    t2 = Thread(target = threadTask)
    
    t1.start()
    t2.start()
    
    t1.join()
    t2.join()

for i in range(10):
    main()
    print(f"{i} -> {globVar}")

0 -> 1673981
1 -> 1559361
2 -> 1520942
3 -> 1740946
4 -> 1608316
5 -> 1609151
6 -> 1695534
7 -> 1588607
8 -> 1692432
9 -> 1692081


In [9]:
from threading import Thread, active_count, main_thread, current_thread
from os import getpid
from time import sleep

def threadName():
    sleep(2)
    print(f"This is thread: {current_thread().name}")


if __name__ == "__main__":
    print("ID of process running main program:", getpid()) 

    th = Thread(target = threadName, name = "th")
    th.start()

    print("Current thread count = ", active_count())
    print(f"Thread {th.getName()} is {th.is_alive()}")
    print()

    th.join()

    print()
    print(f"Thread {th.getName()} is {th.is_alive()}")
    print("Current thread count = ", active_count())
    print()

    print(f"Main thread's name = {main_thread().name}")
    print(f"Current thread's name in main = {current_thread().name}")

ID of process running main program: 25032
Current thread count =  9
Thread th is True

This is thread: th

Thread th is False
Current thread count =  8

Main thread's name = MainThread
Current thread's name in main = MainThread


### Number of threads on interactive kernels like jupyter would likely be higher
This is what the output to the above program would look like when run as a normal python file:

```
ID of process running main program: 12966
Current thread count =  2
Thread th is True

This is thread: th

Thread th is False
Current thread count =  1

Main thread's name = MainThread
Current thread's name in main = MainThread
```

In [10]:
t = Thread(target = threadName)
print(t.getName()) # Returns the default name assigned to the thread during its creation

t.start()
t.join()

Thread-32
This is thread: Thread-32


# Inter Process Communication (IPC)
A process can be of two types:
1. Independent process.
2. Co-operating process.

An independent process is not affected by the execution of other processes while
a co-operating process can be affected by other executing processes. Though one
can think that those processes, which are running independently, will execute
very efficiently, in reality, there are many situations when co-operative nature can
be utilised for increasing computational speed, convenience and modularity. Inter
process communication (IPC) is a mechanism which allows processes to
communicate with each other and synchronize their actions. The communication
between these processes can be seen as a method of co-operation between
them. Processes can communicate with each other through both:

1. Shared Memory
2. Message passing

The Figure 1 below shows a basic structure of communication between processes via the shared memory method and via the message passing method.

<center>
    <img src="https://media.geeksforgeeks.org/wp-content/uploads/1-76.png"/>
</center>

In [11]:
from multiprocessing import Process, Array, Value

resultLocal = [] # Local to the current Process. Hence any changes made to this outside this process won't be reflected below

def squareList(a, result, squareSum):
    for i, num  in enumerate(a):
        result[i] = num ** 2
        resultLocal.append(result[i])
    squareSum.value = sum(result)
    
    print("In Process P1")
    print(f"Shared Variable: {result[:]}")
    print(f"Local Variable: {resultLocal}")
    print("Sum of squares = ", squareSum.value)
    print("-------------------------------")

if __name__ == "__main__":
    a = range(1, 6)
    result = Array('i', 5) # Shared between multiple Processes (C style IPC)
    squareSum = Value('i') # Shared between multiple Processes (C style IPC)
    p1 = Process(target = squareList, args = (a, result, squareSum), name = "P1")
    p1.start()
    p1.join()
    print("\nIn Main Process")
    print(f"Shared Variable: {result[:]}")      # Changes reflected
    print(f"Local Variable: {resultLocal}")     # Changes not reflected
    print("Sum of squares = ", squareSum.value) # Changes reflected

In Process P1
Shared Variable: [1, 4, 9, 16, 25]
Local Variable: [1, 4, 9, 16, 25]
Sum of squares =  55
-------------------------------

In Main Process
Shared Variable: [1, 4, 9, 16, 25]
Local Variable: []
Sum of squares =  55


# Multiprocessing
* Ability of a system to support more than one processor to execute instructions at the same time.
* It is a mode of operation in which two or more processors in a computer simultaneously process two or more different portions of the same program (set of instructions). 
* Applications in a multiprocessing system are broken into smaller routines that run independently. 
* The operating system allocates these threads to the processors, improving performance of the system.
* For example, different processors may be used to manage memory storage, data communications, or arithmetic functions.
* In Python, this is used to launch multiple processes which the OS can then schedule to different logical processors (threads).
- ***Multiprocessing***: More than one process is running on a single processor.
- ***Parallel Processing***: One Process (broken into different parts) running on multiple processors.

## Need for Multiprocessing
Consider a computer system with a single processor. If it is assigned several
processes at the same time, it will have to interrupt each task and switch briefly
to another (context switch), to keep all of the processes going.
This situation is just like a chef working in a kitchen alone. He has to do several
tasks like baking, stirring, kneading dough, etc.

----

## Stackoverflow
### Multi Processing
Multiprocessing is the use of two or more central processing units (CPUs) within a single computer system. 
The term also refers to the ability of a system to support more than one processor and/or the ability to allocate tasks between them.

### Parallel Processing
In computers, parallel processing is the processing of program instructions by dividing them among multiple processors with the objective of running a program in less time. 
In the earliest computers, only one program ran at a time.

In [12]:
from multiprocessing import Process

def cube(num):
    print(f"Cube of the given number is {num ** 3}")

def square(num):
    print(f"Square of the given number is {num ** 2}")

if __name__ == "__main__":
    p1 = Process(target = cube, args = (10,))
    p2 = Process(target = square, args = (2,))

    p1.start()
    p2.start()

    p1.join()
    p2.join()

    print("Both processes finished")

Square of the given number is 4
Cube of the given number is 1000
Both processes finished


Output obtained when run as a normal Python file:

```
Cube of the given number is 1000
Square of the given number is 4
Both processes finished
```

In [13]:
from multiprocessing import Process, current_process
from os import getpid

def printPID():
    print("\nCurrent Process: ", current_process())
    print("PID: ", getpid())

if __name__ == "__main__":
    print("Main process id", getpid())

    p1 = Process(target = printPID) 
    p2 = Process(target = printPID)

    p1.start()
    p2.start()
 
    p1.join()
    p2.join()

    print("\nBoth processes finished execution!")
    print("p1 status is alive?:", p1.is_alive())
    print("p2 status is alive?:", p2.is_alive())

Main process id 25032

Current Process:  <Process name='Process-5' parent=25032 started>
PID:  25161

Current Process:  <Process name='Process-4' parent=25032 started>
PID:  25160

Both processes finished execution!
p1 status is alive?: False
p2 status is alive?: False


# Class 10 (13.10.2021)

## Lock Mechanisms
Lock or a mutex is a synchronization mechanism for enforcing limits on access to a resource in an environment with multiple threads in execution.

In [14]:
from multiprocessing import Process
from time import sleep

def deposit(total):
    for i in range(100):
        sleep(0.01)
        total += 5
    return total

def withdraw(total):
    for i in range(100):
        sleep(0.01)
        total -= 5
    return total

if __name__ == "__main__":
    total = 500
    print(f"Initial value = {total}")
    total = deposit(total)
    print(f"Value after deposit = {total}")
    total = withdraw(total)
    print(f"Value after withdra = {total}")

Initial value = 500
Value after deposit = 1000
Value after withdra = 500


In [15]:
# Multiprocessing with no lock mechanisms (No Atomicity)
from multiprocessing import Process, Value
from time import sleep

def deposit(total):
    for i in range(100):
        sleep(0.01)
        total.value += 5
    return total

def withdraw(total):
    for i in range(100):
        sleep(0.01)
        total.value -= 5
    return total

if __name__ == "__main__":
    # Value class returns a "ctypes" object which is present in the Shared Memory by default.
    # It is synchronized using RLock Mechanism (a locking mechanism part of the multiprocessing library).
    # It is used to share information between processes (Inter Process Communication, IPC).
    # Synchronized manager server ?? (Look up)
    total = Value('i', 500) # "ctypes" Int Object (i stands for int) with a value of 500
    print(f"Total Object = {total}")
    print(f"Initial Value = {total.value}")

    add = Process(target = deposit, args = (total,))
    sub = Process(target = withdraw, args = (total,))

    add.start()
    sub.start()

    add.join()
    sub.join()

    print(f"Total = {total.value}")

Total Object = <Synchronized wrapper for c_int(500)>
Initial Value = 500
Total = 500


In [16]:
from multiprocessing import Process, Value, Lock
from time import sleep

# With lock
lock = Lock() # Lock Object
print(f"Lock Object = {lock}")

def deposit(total, lock):
    for i in range(100):
        sleep(0.01)
        lock.acquire()
        total.value += 5
        lock.release()
    return total

def withdraw(total, lock):
    for i in range(100):
        sleep(0.01)
        lock.acquire()
        total.value -= 5
        lock.release()
    return total

if __name__ == "__main__":
    total = Value('i', 500)
    print(f"Total Object = {total}")
    print(f"Initial Value = {total.value}")

    add = Process(target = deposit, args = (total, lock))
    sub = Process(target = withdraw, args = (total, lock))

    add.start()
    sub.start()

    add.join()
    sub.join()

    print(f"Total = {total.value}")

Lock Object = <Lock(owner=None)>
Total Object = <Synchronized wrapper for c_int(500)>
Initial Value = 500
Total = 500


# Race Conditions
A race condition occurs when two or more threads can access shared data and they try to change it at the same time. Because the thread scheduling algorithm can swap between threads at any time, you don't know the order in which the threads will attempt to access the shared data. Therefore, the result of the change in data is dependent on the thread scheduling algorithm, i.e. both threads are "racing" to access/change the data.

In [17]:
from threading import Thread
import random
import sys
from time import sleep

def f1(s):
    for i in range(0, len(s)):
        print(s[i], end = '')
        sys.stdout.flush() # Flush the data buffer
        sleep(random.random()*3)
        print(s[i], end = '')
        sys.stdout.flush()
        sleep(random.random()*3)

t1 = Thread(target = f1, args = ('ABCDEFGH',))
t2 = Thread(target = f1, args = ('abcdefgh',))

t1.start()
t2.start()

t1.join()
t2.join()

AaaABbBCCbcDcDEEdFdFeGGefHHfgghh

In [18]:
from threading import Thread, Lock
import random
import sys
from time import sleep

lock = Lock()

def f1(s):
    for i in range(0, len(s)):
        lock.acquire()
        print(s[i], end = '')
        sys.stdout.flush() # Flush the data buffer
        sleep(random.random()*3)
        print(s[i], end = '')
        sys.stdout.flush()
        lock.release()
        sleep(random.random()*3)

t1 = Thread(target = f1, args = ('ABCDEFGH',))
t2 = Thread(target = f1, args = ('abcdefgh',))

t1.start()
t2.start()

t1.join()
t2.join()

AAaabbBBccCCDDddEEeeFFffggGGhhHH

# Class 11 (20.10.2021)

# Semaphores
* One of the oldest synchronization primitives in the history of Computer Science.
* Invented by Dutch scientist Dijkstra.
* **p()** {Wait/Acquire} and **v()** {Signal/Release}.

In [19]:
from threading import Semaphore

# This is not a bounded Semaphore.
sem = Semaphore(5) # Allows 5 threads to acquire it, i.e 5 threads can be in the critical section at a time

print(sem._value)

print(sem.acquire())

print(sem._value)

print(sem.release())
print(sem.release()) # Can release without acquiring as well (not ideal in some cases, hence use BoundedSemaphore)

print(sem._value)

5
True
4
None
None
6


In [20]:
from threading import BoundedSemaphore

bsem = BoundedSemaphore(10)

print(bsem.acquire())
print(bsem._value)

print(bsem.release())
print(bsem.release()) # Cannot release without acquiring

print(bsem._value)

True
9
None


ValueError: Semaphore released too many times

In [21]:
sem = Semaphore(1)
print(sem.acquire())
# By default, the acquire method for Semaphores is blocking. Hence, it blocks the thread trying to acquire it when its value is 0
print(sem.acquire()) # Check if this is busy wait...
print("Acquired!")

True


KeyboardInterrupt: 

In [22]:
sem = Semaphore(1)
print(sem.acquire(blocking = False))
print(sem.acquire(blocking = False)) # When Blocking is set to False, It returns False immediately if the Semaphore cannot be acquired
print("Acquired!")

True
False
Acquired!


In [23]:
from threading import Semaphore, Thread
from time import  sleep

sem = Semaphore() # Default value of Semaphore count is 1
print(sem, sem._value)

# Color coding for ease of visualization
RED = "\033[91m"
GREEN = "\033[92m"
RESET = "\033[0m"

def f1():
    print("Starting Function f1")
    sem.acquire()
    for loop in range(1, 5):
        print(f"{loop}) Function f1 in loop")
        sleep(0.02)
    sem.release()
    print("Function f1 done")

def f2():
    print("Starting Function f2")
    while not sem.acquire(blocking = False):
        print(f"\n{RED}Function f2: Semaphore unavailable{RESET}\n")
        sleep(0.05)
    else:
        print(f"\n{GREEN}Function f2: Semaphore acquired{RESET}\n")
        for loop in range(1, 5):
            print(f"{loop}) Function f2 in loop")
            sleep(0.02)
    sem.release()

t1 = Thread(target = f1)
t2 = Thread(target = f2)

t1.start()
t2.start()

t1.join()
t2.join()

<threading.Semaphore object at 0x7f4b3c6587c0> 1
Starting Function f1
1) Function f1 in loop
Starting Function f2

Function f2: Semaphore unavailable

2) Function f1 in loop
3) Function f1 in loop

Function f2: Semaphore unavailable

4) Function f1 in loop
Function f1 done

Function f2: Semaphore acquired

1) Function f2 in loop
2) Function f2 in loop
3) Function f2 in loop
4) Function f2 in loop


# Difference between Semaphore, BoundedSemaphore

In [24]:
from threading import Semaphore, BoundedSemaphore

# Usually, you create a Semaphore that will allow a certain number of threads
# into a section of code. This one starts at 5.
s1 = Semaphore(5)

# When you want to enter the section of code, you acquire it first.
# That lowers it to 4. (Four more threads could enter this section.)
s1.acquire()

# Then you do whatever sensitive thing needed to be restricted to five threads.
# When you're finished, you release the semaphore, and it goes back to 5.
s1.release()

# That's all fine, but you can also release it without acquiring it first.
s1.release()

# The counter is now 6! That might make sense in some situations, but not in most.
print(s1._value) # => 6

# If that doesn't make sense in your situation, use a BoundedSemaphore.
s2 = BoundedSemaphore(5) # Start at 5.
s2.acquire() # Lower to 4.
s2.release() # Go back to 5.
print(s2._value)

try:
    s2.release() # Try to raise to 6, above starting value.
    print(s2._value)
except ValueError:
    print('As expected, it complained!') 

6
5
As expected, it complained!


# Producer Consumer Problem (PCP)
Helps us understand concurrency.
* Producer produces stuff and pushes it to a buffer.
* Consumer will consume the stuff from the buffer.

In [28]:
from threading import Thread, Lock
from time import  sleep
from random import choice, random

queue = []
lock = Lock()

# Color coding the output for easy visualization
RED = "\033[91m"
GREEN = "\033[92m"
RESET = "\033[0m"
C1 = "\033[93m"
C2 = "\033[94m"
C3 = "\033[95m"

class Producer(Thread):
    def run(self):
        nums = range(5)
        global queue
        for i in range(10): # Run the Condition.py file for non terminating simulation of the PCP
            num = choice(nums)
            with lock:
                queue.append(num)
                print(f"{C1}Producer: {C3}Produced {num}{RESET}")
            sleep(random())

class Consumer(Thread):
    def run(self):
        global queue
        for i in range(10):
            with lock:
                if not queue: 
                    print(f"{C2}Consumer: {RED}Queue Empty{RESET}")
                else: # If this else condition is removed, the consumer will try to pop from an empty buffer leading to an IndexError
                    num = queue.pop(0)
                    print(f"{C2}Consumer: {GREEN}Consumed {num}{RESET}")
            sleep(random())

Producer().start()
Consumer().start()          

Producer: Produced 0
Consumer: Consumed 0
Consumer: Queue Empty
Producer: Produced 4
Consumer: Consumed 4
Producer: Produced 0
Producer: Produced 2
Consumer: Consumed 0
Producer: Produced 4
Producer: Produced 0
Consumer: Consumed 2
Consumer: Consumed 4
Producer: Produced 3
Consumer: Consumed 0
Consumer: Consumed 3
Producer: Produced 2
Consumer: Consumed 2
Producer: Produced 3
Producer: Produced 0
Consumer: Consumed 3


# Class 12 (25.10.2021)

# Condition Object
* Allows one or more threads to wait until notified by another thread. (Prevents the consumer from trying to consume from an empty buffer in the previous case)
* This can be used to prevent concurrency bugs.
* Conditions are associated with locks and have two methods --> ***acquire()*** and ***release()***

In [29]:
from threading import Thread, Condition
from time import  sleep
from random import choice, random

simulationLength = range(20)
queue = []
maxNum = 5
condition = Condition()

# Color coding the output for easy visualization
RED = "\033[91m"
GREEN = "\033[92m"
RESET = "\033[0m"
C1 = "\033[93m"
C2 = "\033[94m"
C3 = "\033[95m"

class Producer(Thread):
    def run(self):
        nums = range(5)
        global queue
        for i in simulationLength:
            num = choice(nums)
            condition.acquire() # "with condition" can be used instead of explicitly using the acquire and release methods
            if len(queue) == maxNum: 
                print(f"{C1}Producer: {RED}Queue Full{RESET}")
                condition.wait() # Wait for Consumer to notify after it has consumed something and made room for new products in the queue
            queue.append(num)
            print(f"{C1}Producer: {C3}Produced {num}{RESET}")
            condition.notify() # Notify the consumer that a new product has been produced
            condition.release()
            sleep(random())

class Consumer(Thread):
    def run(self):
        global queue
        for i in simulationLength:
            condition.acquire()
            if not queue: 
                print(f"{C2}Consumer: {RED}Queue Empty{RESET}")
                condition.wait() # Wait for Producer to add something the queue
            num = queue.pop(0)
            print(f"{C2}Consumer: {GREEN}Consumed {num}{RESET}")
            condition.notify() # Notify the producer to add something to the queue
            condition.release()
            sleep(random())

Producer().start()
Consumer().start()               

Producer: Produced 1
Consumer: Consumed 1
Producer: Produced 4
Consumer: Consumed 4
Producer: Produced 3
Consumer: Consumed 3
Consumer: Queue Empty
Producer: Produced 0
Consumer: Consumed 0
Producer: Produced 4
Producer: Produced 2
Consumer: Consumed 4
Consumer: Consumed 2
Producer: Produced 2
Producer: Produced 1
Consumer: Consumed 2
Consumer: Consumed 1
Producer: Produced 2
Consumer: Consumed 2
Consumer: Queue Empty
Producer: Produced 0
Consumer: Consumed 0
Consumer: Queue Empty
Producer: Produced 3
Consumer: Consumed 3
Consumer: Queue Empty
Producer: Produced 0
Consumer: Consumed 0
Consumer: Queue Empty
Producer: Produced 3
Consumer: Consumed 3
Consumer: Queue Empty
Producer: Produced 1
Consumer: Consumed 1
Consumer: Queue Empty
Producer: Produced 3
Consumer: Consumed 3
Producer: Produced 3
Producer: Produced 0
Consumer: Consumed 3
Producer: Produced 0
Producer: Produced 4
Consumer: Consumed 0
Consumer: Consumed 0
Producer: Produced 2
Consumer: Consumed 4
Consumer: Consumed 2


# Explanation
1. The Consumer checks if the buffer is empty before consuming.
2. If the buffer is empty, it calls **wait()** on the condition instance.
3. **wait()** blocks the consumer and also releases the lock associated with the condition, i.e it loses hold of the lock.
4. Now unless consumer is notified, it will not run.
5. Now, the Producer can acquire the lock as it was released by the Consumer.
6. Producer puts data into the buffer and calls **notify()** on the condition instance.
7. Once **notify()** call is made on condition, Consumer wakes up. But waking up doesn’t mean it starts executing.
8. **notify()** does not release the lock. Even after **notify()**, lock is still held by the Producer.
9. Producer explicitly releases the lock by using **condition.release()**.
10. Consumer starts running again and it'll consume the data in the buffer.
11. No IndexError will be raised.

In [30]:
dir(Producer)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_bootstrap',
 '_bootstrap_inner',
 '_delete',
 '_initialized',
 '_reset_internal_locks',
 '_set_ident',
 '_set_native_id',
 '_set_tstate_lock',
 '_stop',
 '_wait_for_tstate_lock',
 'daemon',
 'getName',
 'ident',
 'isDaemon',
 'is_alive',
 'join',
 'name',
 'native_id',
 'run',
 'setDaemon',
 'setName',
 'start']